# INCA Parser - Quick start
The INCA parser introduce a python wrapper to the Matlab application INCA. INCA can apply multiple 13C-MFA great algorithms. Unfortunately, it is meant to be used through a GUI and therefore it is laborious to handle many samples and measurements and is hard in incorporate into high throughput pipelines. The INCA parser operates on both sides of INCA it helps input model and data into INCA, it can run INCA algorithms and finally provides a workflow to export the results from INCA after execution of algorithms. This will be a very quick walk through of how to use the INCA Parser to give interested users an overview of the capabilities.

In [1]:
import pandas as pd
import pathlib
import incawrapper
import ast


The input data to the INCA parser is given as pandas dataframes. These has to comply to specified dataschemas (See XX for a thorough walk through of the requirements). The minimum inputs to run MFA in INCA is 
- Reaction data
- Tracer data

For stationary 13C-MFA least one of
- Flux measurements
- MS measurements

For Isotopically Non-stationary 13C-MFA additionally measurements of the metabolite pool are required.

Let's load some data to see the format of the input. All input data is validated for column name and data types, thus it will raise an error if either of those are incorrect.

In [2]:
data_folder = pathlib.Path("./Literature data/simple model")
tracers_data = pd.read_csv(data_folder / "tracers.csv", 
   converters={'atom_mdv':ast.literal_eval, 'atom_ids':ast.literal_eval} # a trick to read lists from csv
)
reactions_data = pd.read_csv(data_folder / "reactions.csv")
flux_data = pd.read_csv(data_folder / "flux_measurements.csv")
ms_data = pd.read_csv(data_folder / "ms_measurements.csv", 
   converters={'labelled_atom_ids': ast.literal_eval} # a trick to read lists from csv
)

We use a simple toy model with 5 reactions [1,2]. In this data set we have two experiments exp1 and exp2. They were conducted with different tracers. For exp1 we have both flux and MS measurements, while for exp2 the data set only contains MS measurements. We will quick view the input data, again for a detail description see XX.

In [3]:
reactions_data.head()

,model,rxn_id,rxn_eqn
0,simple_model,R1,A (abc) -> B (abc)
1,simple_model,R2,B (abc) <-> D (abc)
2,simple_model,R3,B (abc) -> C (bc) + E (a)
3,simple_model,R4,B (abc) + C (de) -> D (bcd) + E (a) + E (e)
4,simple_model,R5,D (abc) -> F (abc)


In [4]:
tracers_data.head()

,experiment_id,met_id,tracer_id,atom_ids,ratio,atom_mdv,enrichment
0,exp1,A,[2-13C]A,[2],1.0,"[0, 1]",1
1,exp2,A,"[1,2-13C]A","[1, 2]",0.5,"[0.05, 0.95]",1


In [5]:
flux_data.head()

,experiment_id,rxn_id,flux,flux_std_error
0,exp1,R1,10.0,0.00001


In [6]:
ms_data.head()

,experiment_id,met_id,ms_id,measurement_replicate,labelled_atom_ids,unlabelled_atoms,mass_isotope,intensity,intensity_std_error,time
0,exp1,F,F1,1,"[1, 2, 3]",NaN,0,0.0001,0.000002,0
1,exp1,F,F1,1,"[1, 2, 3]",NaN,1,0.8008,0.016016,0
2,exp1,F,F1,1,"[1, 2, 3]",NaN,2,0.1983,0.003966,0
3,exp1,F,F1,1,"[1, 2, 3]",NaN,3,0.0009,0.000018,0
4,exp2,F,F1,1,"[1, 2, 3]",NaN,0,0.0002,0.000002,0


With the data loaded we are ready to create the INCA script that specifies the model, data and algorithms that we wish to apply. We will specify that the estimation algorithm should do 5 restarts and that INCA should not do naturel abundance correction. Finally, specify that we wish to run three of INCA's algorithms: estimate, simulate and continuate and that the model including the results should be saved in a file called simple_model_quickstart.mat in the current working directory.

In [7]:
output_file = pathlib.Path("./Literature data/simple model/simple_model_quikstart.mat")
script = incawrapper.create_inca_script_from_data(reactions_data, tracers_data, flux_data, ms_data, experiment_ids=["exp1"])
script.add_to_block("options", incawrapper.define_options(fit_starts=5,sim_na=False))
script.add_to_block("runner", incawrapper.define_runner(output_file, run_estimate=True, run_simulation=True, run_continuation=True))

In [8]:
print(script)

clear functions

% REACTION BLOCK
% Create reactions
r = [...
reaction('A (abc) -> B (abc)', ['id'], ['R1']),...
reaction('B (abc) <-> D (abc)', ['id'], ['R2']),...
reaction('B (abc) -> C (bc) + E (a)', ['id'], ['R3']),...
reaction('B (abc) + C (de) -> D (bcd) + E (a) + E (e)', ['id'], ['R4']),...
reaction('D (abc) -> F (abc)', ['id'], ['R5']),...
];

% TRACERS BLOCK
% define tracers used in exp1
t_exp1 = tracer({...
'[2-13C]A: A @ 2',...
});
t_exp1.frac = [1.0 ];
t_exp1.atoms.it(:,1) = [0,1];


% FLUXES BLOCK

% define flux measurements for experiment exp1
f_exp1 = [...
data('R1', 'val', 10.0, 'std', 1e-05),...
];


% MS_FRAGMENTS BLOCK

% define mass spectrometry measurements for experiment exp1
ms_exp1 = [...
msdata('F1: F @ 1 2 3'),...
];

% define mass spectrometry measurements for experiment exp1
ms_exp1{'F1'}.idvs = idv([[0.0001;0.8008;0.1983;0.0009]], 'id', {'exp1_F1_0_0_1'}, 'std', [[2e-06;0.016016;0.003966;1.8e-05]], 'time', [0.0])


% POOL_SIZES BLOCK


% EXPERIMENTAL_DATA B

Now the script is ready to run in matlab.

In [9]:
import dotenv
inca_directory = pathlib.Path(dotenv.get_key(dotenv.find_dotenv(), "INCA_base_directory"))
incawrapper.run_inca(script, INCA_base_directory=inca_directory)

INCA script saved to /var/folders/z6/mxpxh4k56tv0h0ff41vmx7gdwtlpvp/T/tmpefnuuvn2/inca_script.m.
Starting MATLAB engine...
 
ms_exp1 = 1x1 msdata object
 
fields: atoms  id  [idvs]  more  on  state  
 
F1
 
 
m = 1x1 model object
 
fields: [expts]  [mets]  notes  [options]  [rates]  [states]  
 
	5 reactions (6 fluxes)                                  
	6 states (3 balanced, 1 source, 2 sink and 0 unbalanced)
	6 metabolites                                           
	1 experiments                                           
 

                                         Directional 
 Iteration      Residual     Step-size    derivative        Lambda
     0       9.99482e+11
     1       9.97429e+11      0.000985     -1.05e+12       2307.44
     2       8.39378e+07             1     -2.59e+09       2307.44
     3       3.20925e+07             1     -6.85e+06       769.148
     4       1.20656e+07             1     -4.66e+06        759.12
     5            673446             1      -1.6e+06  

We can now read the results from INCA using the `INCAResults` object.

In [11]:
res = incawrapper.INCAResults(output_file)

Typically, the first thing we want to inspect is the fitted fluxes. They can be displayed as a Pandas dataframe.

In [12]:
res.fitdata.fitted_parameters

,type,id,eqn,val,std,lb,ub,unit,free,alf,chi2s,cont,cor,cov,vals,base
0,Net flux,R1,A -> B,10.000000,0.000010,9.99998,10.00002,[],0,0.05,"[141.10067495553355, 140.33238319091117, 139.5...",0,"[1.0000000000000002, 8.947243325066131e-05, 2....","[1.0000002822335707e-10, 6.084447182197472e-11...","[9.99997852967027, 9.999980400360151, 9.999982...",{'id': []}
1,Net flux,R2 net,B <-> D,6.084460,0.068004,5.947701,6.218202,[],1,0.05,"[141.0570054790283, 140.2904927224016, 139.524...",0,"[8.947241284322896e-05, 1.0000000000000002, 0....","[6.084445794418691e-11, 0.004624487730732776, ...","[5.935231731817345, 5.948458357367386, 5.96306...",{'id': []}
2,Exch flux,R2 exch,B <-> D,6.622541,0.330830,6.00107,7.352862,[],1,0.05,"[140.34961899282376, 139.5777181227731, 138.80...",0,"[2.001768917398014e-05, 0.9284327082119621, 1....","[6.622453627147529e-11, 0.02088753405517206, 0...","[5.999808683642599, 6.0609390135975545, 6.1315...",{'id': []}
3,Net flux,R3,B -> C + E,1.957770,0.034002,1.890899,2.026149,[],1,0.05,"[141.07978895781474, 140.31198477385965, 139.5...",0,"[5.7578676443448344e-05, -0.999999989187987, -...","[1.957778513958508e-11, -0.0023122438349441523...","[1.8847369172037767, 1.8910756941864049, 1.898...",{'id': []}
4,Net flux,R4,B + C -> D + E + E,1.957770,0.034002,1.890899,2.026149,[],0,0.05,"[141.07978895781474, 140.31198477385965, 139.5...",0,"[5.7578676443448344e-05, -0.999999989187987, -...","[1.957778513958508e-11, -0.0023122438349441523...","[1.8847369172037767, 1.8910756941864049, 1.898...",{'id': []}
5,Net flux,R5,D -> F,8.042230,0.034002,7.973851,8.109101,[],0,0.05,"[141.0570054656954, 140.2904927062726, 139.524...",0,"[0.00023652349632564553, 0.9999999891879908, 0...","[8.0422243083772e-11, 0.002312243895788624, 0....","[7.967615864363539, 7.974229177314326, 7.98153...",{'id': []}
6,Norm,exp1 F1 exp1_F1_0_0_1 norm,[],0.985718,0.016453,[],[],[],1,0.05,[],0,"[-9.489258988154049e-11, 0.582260040985265, 0....","[-1.5612511283791264e-17, 0.000651462177349523...",[],{'id': []}


In [13]:
res.fitdata.get_goodness_of_fit()

Fit accepted: False
Confidence level: 0.05
Chi-square value (SSR): 136.4909243702529
Expected chi-square range: [9.82069117e-04 5.02388619e+00]


In [14]:
import incawrapper.visualization as incaviz
incaviz.plot_norm_probplot(res)

ModuleNotFoundError: No module named 'sklearn'